In [6]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import csv

# File path for the CSV file
output_csv_file = '/Users/ashwinkashyap/Downloads/output_dataset.csv'

nlevels = 8
scaleFactor = 1.2
EDGE_THRESHOLD = 19
PATCH_SIZE = 31
mvScaleFactor = [None] * nlevels

mvScaleFactor[0] = 1.0

for i in range(1, nlevels):
    mvScaleFactor[i]=mvScaleFactor[i-1]*scaleFactor

# create SIFT feature extractor
sift = cv2.SIFT_create()

def compute_features_pp(filename, dnfname):

    image = cv2.imread(dnfname)
    # Initialize mvImagePyramid
    mvImagePyramid = [None] * nlevels

    for level in range(0, nlevels):
        if level == 0:
            mvImagePyramid[0] = cv2.copyMakeBorder(image, EDGE_THRESHOLD, EDGE_THRESHOLD, EDGE_THRESHOLD, EDGE_THRESHOLD,
                                           cv2.BORDER_REFLECT_101)
        else:
            scale = scaleFactor ** level
            sz = (int(image.shape[1] / scale), int(image.shape[0] / scale))
            resized = cv2.resize(mvImagePyramid[level-1], sz, interpolation=cv2.INTER_LINEAR)
            mvImagePyramid[level] = cv2.copyMakeBorder(resized, EDGE_THRESHOLD, EDGE_THRESHOLD, EDGE_THRESHOLD, EDGE_THRESHOLD,
                                                   cv2.BORDER_REFLECT_101 + cv2.BORDER_ISOLATED)
        
        # detect features from the image
        keypoints, descriptors = sift.detectAndCompute(mvImagePyramid[level], None)
        

        for keypoint in keypoints:
            keypoint.octave = level
            keypoint.size = PATCH_SIZE*mvScaleFactor[level]
            keypoint.response = round(keypoint.response * 1000)
            
            # Open the CSV file in write mode
            with open(output_csv_file, 'a', newline='') as file:
                # Create a CSV writer object
                writer = csv.writer(file)
                # Write the data to the CSV file
                row = [str(val) for val in [filename, keypoint.pt[0], keypoint.pt[1], keypoint.size, keypoint.angle, keypoint.response, keypoint.octave, keypoint.class_id]]
                writer.writerow(row)


# Specify the directory path
directory = "/Users/ashwinkashyap/Dissertation/Code/rgbd_dataset_freiburg1_floor/rgb"

# Iterate through the directory
for filename in os.listdir(directory):
    # Check if the file has an image extension
    if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
        dnfname = directory+'/'+filename
        compute_features_pp(filename, dnfname)
print(f"Data has been written to '{output_csv_file}'.")

KeyboardInterrupt: 